In [ ]:
!pip install python-docx

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from docx import Document
doc = Document("/content/links from clirnet (3).docx")
links = []
for para in doc.paragraphs:
    text = para.text.strip()
    if text.startswith("https"):
        links.append(text)
print("Extracted links:", links)

In [ ]:
!pip install trafilatura

In [ ]:
!pip install beautifulsoup4 requests pandas lxml keybert spacy pandas scikit-learn transformers torch

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import trafilatura
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from transformers import pipeline

In [ ]:
import pandas as pd
from keybert import KeyBERT
import spacy

In [ ]:

# Step 2: Scrape raw text with BeautifulSoup
def scrape_text(url):
    try:
        r = requests.get(url, timeout=15)
        soup = BeautifulSoup(r.text, "lxml")
        paragraphs = [p.get_text(strip=True) for p in soup.find_all("p")]
        return " ".join(paragraphs)
    except Exception:
        return ""

rows = []
for link in links:
    text = scrape_text(link)
    rows.append({"link": link, "raw_text": text})

df_raw = pd.DataFrame(rows)

# Step 3: Hugging Face zero-shot classification
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ["Agriculture & Environment", "Medicine & Health", "Space Science", "Global Development", "Technology", "General"]

topics = []
for text in df_raw["raw_text"]:
    if not text.strip():
        topics.append("General")
        continue
    result = classifier(text, candidate_labels)
    topics.append(result["labels"][0])  # highest scoring topic

df_raw["predicted_topic"] = topics

# Optional: Summarization
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summaries = []
for text in df_raw["raw_text"]:
    if not text.strip():
        summaries.append("")
        continue
    result = summarizer(text[:1000])  # limit length for performance
    summaries.append(result[0]["summary_text"])

df_raw["summary"] = summaries

# Step 4: Show enriched dataset
print(df_raw[["link","predicted_topic","summary"]].head())

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]